In [14]:
! pip install chromadb

In [15]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
load_dotenv() # not needed for loacal modals.. like sentence transformers

False

In [16]:
! pip install sentence-transformers

In [18]:
from sentence_transformers import SentenceTransformer
# use sentence transformer embedding function
model_name= 'sentence-transformers/all-MiniLM-L6-v2'

embedding_fn= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\think\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\think\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
#  initialize the chroma client.


# using the persistant client to store and load the data autoamatically if it exists

client= chromadb.PersistentClient(path='CHROMA_persistant_storage')
collection_name= "documenet_qa_collection"
collection= client.get_or_create_collection(
    name= collection_name,
    embedding_function= embedding_fn,
)

In [20]:
# load all the documents [txt docs]
def load_documents_from_directory(director_path):
    documents=[]
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            with open(os.path.join(directory_path, filename)) as file:
                documents.append({
                    'id':filename,
                    'text': file.read(),
                    'metadata': {'filename': filename}
                })
    return documents

In [33]:
# split the text.
def split_text(text, chunk_size= 1000, chunk_overlap=20):
    chunks=[]
    start=0
    while start<len(text):
        end= start + chunk_size
        chunks.append(text[start:end])
        start= end- chunk_overlap
    return chunks

In [36]:
# create the chunks of docs using the split_text function
def create_document_chunks(documents, chunk_size=1000, chunk_overlap=20):
    chunked_documents=[]
    for doc in documents:
        doc_chunks=split_text(doc['text'], chunk_size, chunk_overlap)
        print("Splitting Document:", doc['metadata']['filename'], "into chunks")
        for i, chunk in enumerate(doc_chunks):
            chunked_documents.append(
                {
                    'id': f"{doc['id']}_chunk_{i}",
                    "chunk": f"{chunk}" ,
                    'metadata': {
                        'filename': doc['metadata']['filename'],
                        'chunk_index': i
                    }
                    
                }
            )
    return chunked_documents


🔁 Example
Let’s say:
```
text = "abcdefghij"

chunk_size = 4

chunk_overlap = 2

How it chunks:

Iteration	start	end	chunk
1	0	4	'abcd'
2	2	6	'cdef'
3	4	8	'efgh'
4	6	10	'ghij'

Result: ['abcd', 'cdef', 'efgh', 'ghij']

Note the 2-character overlap (cd, ef, gh) between each chunk.
```

In [28]:
directory_path= './news_articles'
documents=load_documents_from_directory(directory_path)
print(len(documents), "documents loaded")
print(documents[0])

21 documents loaded
{'id': '02.txt', 'text': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television

In [35]:
Doc_chunks_result=create_document_chunks(documents)
print(Doc_chunks_result[0:2])

[{'id': '02.txt_chunk_0', 'chunk': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television Producers

In [40]:
# build the embeddings function
def get_embeddings(text):
    response= embedding_fn([text])
    embedding= response[0]
    print("====Generating Embeddings====")
    return embedding

In [41]:
# call the function over all the chunks
def get_chunk_embeddings(chunked_documents):
    for chunk in chunked_documents:
        chunk['embedding']= get_embeddings(chunk['chunk'])
    return chunked_documents

In [42]:
Embedding_result= get_chunk_embeddings(Doc_chunks_result)

====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Generating Embeddings====
====Genera

In [43]:
print("Total chunks created:", len(Embedding_result))

Total chunks created: 184


In [45]:
print(Embedding_result[0])

{'id': '02.txt_chunk_0', 'chunk': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television Producers 

In [56]:
# insert each embedding into Chroma DB
def add_embeddings_and_chunks_to_Chromacollection(dict):
    for doc in dict:
        print(doc)
        collection.upsert(
            ids= [doc['id']],
            embeddings= [doc['embedding']],
            documents= [doc['chunk']],
            metadatas= doc['metadata']          
        )

In [57]:
add_embeddings_and_chunks_to_Chromacollection(Embedding_result)

{'id': '02.txt_chunk_0', 'chunk': 'In the must-watch final season of “Succession,” Kendall Roy enters a conference room with his siblings. As the scene opens, he takes a seat and declares: “Who will be the successor? Me.”\n\nOf course, that scene didn’t appear on HBO’s hit show, but it’s a good illustration of generative AI’s level of sophistication compared to the real thing. Yet as the Writers Guild of America goes on strike in pursuit of livable working conditions and better streaming residuals, the networks won’t budge on writers’ demands to regulate the use of AI in writers’ rooms.\n\n“Our proposal is that we not be required to adapt something that’s output by AI, and that the output of an AI not be considered writers’ work,” comedy writer Adam Conover told TechCrunch. “That doesn’t entirely exclude that technology from the production process, but it does mean that our working conditions wouldn’t be undermined by AI.”\n\nBut the Alliance of Motion Picture and Television Producers 

In [61]:
# create a function to query our documents
def query_documents(query, n_results=2):
    results= collection.query(
        query_texts=query,
        n_results=n_results
    )
    # extract the relevant chunks
    relevant_chunks=[doc for sublist in results['documents'] for doc in sublist]
    print('====Returning Relevant Chunks====')
    return relevant_chunks

In [67]:
relevant_chunks=query_documents('What is the impact of AI on the job market?', n_results=2)

====Returning Relevant Chunks====


In [63]:
# let us initiate ollama
! pip install ollama

In [65]:
import ollama

def send_to_ollama_with_context(query, context, model_name='gemma3n:latest'):
    context_str = "\n\n".join(context)

    rag_prompt = f"""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Context:
{context_str}

Question:
{query}

Answer:"""

    try:
        response = ollama.chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': rag_prompt,
                }
            ],
            options={
                'temperature': 0.7,
                'max_tokens': 500,
                'top_p': 0.9,
            }
        )
        return response['message']['content']
    except Exception as e:
        return f"Error generating response: {str(e)}"


In [68]:
print("====Sending to Ollama with Context====")
print(send_to_ollama_with_context(
    'What is the impact of AI on the job market?',relevant_chunks))

====Sending to Ollama with Context====
The context suggests that writers are more concerned about AI being used to undermine working conditions rather than the technology itself. There's worry about automation potentially eliminating jobs, including fulfilling ones.  A letter signed by prominent figures calls for a pause on developing AI systems until the risks can be managed.
